# Shared libraries and packages

In [ ]:
import numpy as np
import pylab as p
import matplotlib.pyplot as plt
import pysindy as ps

# "Final" Code

# Draft / Playground space below:

## Clayton Code

## Jackie Code

## Max Code

## Gaurav Code

## Richard Code

In [3]:
discovered_cure_for_measles = False
if discovered_cure_for_measles:
    print("Project complete")
else:
    print("Still have work to do :(")

Still have work to do :(
